In [1]:
#.venvs/python311_conda
import gzip
import pyfastx
home_dir="../"

In [2]:
human_genome_filepath = home_dir+"data/downloads/hg19_latest/hg19.fa.gz"
fa = pyfastx.Fasta(human_genome_filepath)
fa

<Fasta> ../data/downloads/hg19_latest/hg19.fa.gz contains 298 sequences

In [3]:
import re
search = re.compile(r'[^ACGT]').search
def any_char_other_than_ACGT_in_seq(seq):
    """returns True if any other char found in the input seq than {A,C,G,T} (uppercase only).
    """
    return bool(search(seq))

In [36]:
buffer = []
# ~7 secs to iterate over the data
with gzip.open(home_dir+"data/processed/generic_negative_regions.sorted.tsv.gz", 'rt') as f:
    i = 0
    data = []
    for line in f:
        # print(line)
        data.append(line)

        i+=1
        # if i>=1000: break
        if i%10000==0:
            buffer.append(data)
            data = []
            # print(i)
            i=0

        # if len(buffer)==50: break

buffer.append(data) # saving last data
len(buffer)

1328

In [19]:
len(buffer[:44])

44

In [38]:
from joblib import delayed, Parallel

def get_regions_having_Ns(buffer_no, list_of_lines):
    human_genome_filepath = home_dir+"data/downloads/hg19_latest/hg19.fa.gz"
    fa = pyfastx.Fasta(human_genome_filepath)
    # fa
    
    print("buffer_no: ", buffer_no)
    lines_having_Ns = ""
    for line in list_of_lines:
        # print(line)

        # print(line.split(sep="\t"))
        chrom, start, end = line.split(sep="\t")
        start = int(start.strip())
        end = int(end.strip())

        seq = fa[chrom][start:end].seq
        seq = seq.upper()
        # print(i, seq)

        if any_char_other_than_ACGT_in_seq(seq):
            lines_having_Ns += line
    return lines_having_Ns

n_buffers = len(buffer) #[:10] small scale test
with Parallel(n_jobs=230, verbose=1) as parallel:
    list_of_lines_having_Ns = parallel(delayed(get_regions_having_Ns)(i, buffer[i]) for i in range(n_buffers))

print(len(list_of_lines_having_Ns))

print("saving data")
f_out = gzip.open(home_dir+"data/processed/generic_negative_regions_issues.sorted.tsv.gz", 'wt')
for lines_having_Ns in list_of_lines_having_Ns:
    f_out.write(lines_having_Ns) 
f_out.close()

[Parallel(n_jobs=230)]: Using backend LokyBackend with 230 concurrent workers.


buffer_no:  23
buffer_no:  buffer_no: 85 
66
buffer_no:  21
buffer_no:  18
buffer_no: buffer_no:   18563

buffer_no:  180
buffer_no:  67
buffer_no:  1
buffer_no:  buffer_no: 8
 62
buffer_no:  27
buffer_no: buffer_no:  buffer_no:  3
 15164

buffer_no:  5
buffer_no:  29
buffer_no:  37
buffer_no:  16
buffer_no:  143
buffer_no:  152
buffer_no:  7
buffer_no:  68
buffer_no:  211
buffer_no:  34
buffer_no:  6
buffer_no:  51
buffer_no:  2
buffer_no:  17
buffer_no:  207
buffer_no:  103
buffer_no:  87
buffer_no:  11
buffer_no:  13
buffer_no:  50
buffer_no:  60
buffer_no:  133
buffer_no:  147
buffer_no:  55
buffer_no:  20
buffer_no:  105
buffer_no:  122
buffer_no:  44
buffer_no:  196
buffer_no:  101
buffer_no:  86
buffer_no:  77
buffer_no:  14
buffer_no:  28
buffer_no:  71
buffer_no:  227
buffer_no:  135
buffer_no:  168
buffer_no:  30
buffer_no: buffer_no:   199

buffer_no:  12
buffer_no:  193
buffer_no:  41
buffer_no: buffer_no:   82
79
buffer_no:  buffer_no: buffer_no: 47buffer_no: 
  162 74
190

[Parallel(n_jobs=230)]: Done 340 tasks      | elapsed: 33.4min


buffer_no:  460
buffer_no:  461
buffer_no:  463
buffer_no:  462
buffer_no:  465
buffer_no:  464
buffer_no:  466
buffer_no:  468
buffer_no:  467
buffer_no:  469
buffer_no:  471
buffer_no:  470
buffer_no:  472
buffer_no:  473
buffer_no:  474
buffer_no:  475
buffer_no:  476
buffer_no:  477
buffer_no:  478
buffer_no:  479
buffer_no:  481
buffer_no:  480
buffer_no:  483
buffer_no:  482
buffer_no:  485
buffer_no:  484
buffer_no:  487
buffer_no:  486
buffer_no:  488
buffer_no:  494
buffer_no:  489
buffer_no:  490
buffer_no:  491
buffer_no:  495
buffer_no:  493
buffer_no:  492
buffer_no:  496
buffer_no:  499
buffer_no:  497
buffer_no:  502
buffer_no:  501
buffer_no:  498
buffer_no:  500
buffer_no:  505
buffer_no:  512
buffer_no:  511
buffer_no:  509
buffer_no:  504
buffer_no:  503
buffer_no:  513
buffer_no:  506
buffer_no:  510
buffer_no:  507
buffer_no:  521
buffer_no:  508
buffer_no:  518
buffer_no:  516
buffer_no:  515
buffer_no:  517
buffer_no:  527
buffer_no:  523
buffer_no:  519
buffer_n

In [4]:
# time:
f_out = gzip.open(home_dir+"data/processed/generic_negative_regions_issues.sorted.tsv.gz", 'wt')

with gzip.open(home_dir+"data/processed/generic_negative_regions.sorted.tsv.gz", 'rt') as f:
    i, j, buffer = 0, 0, ""
    for line in f:
        # print(line)

        # print(line.split(sep="\t"))
        chrom, start, end = line.split(sep="\t")
        start = int(start.strip())
        end = int(end.strip())

        seq = fa[chrom][start:end].seq
        seq = seq.upper()
        # print(i, seq)

        if any_char_other_than_ACGT_in_seq(seq):
            # print(i, seq)
            j+=1
            buffer += line
            if j==10000: # writing buffer in the disk
                f_out.write(buffer)
                j, buffer = 0, "" # cleaning buffer

        i+=1
        # if i>=1000: break
        if i%10000==0:
            print(i)

f_out.write(buffer) # writing the last buffer
f_out.close()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

KeyboardInterrupt: 